In [1]:
import yahoo_fin.stock_info as yf
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [28]:
nasdaq_ticker = pd.read_csv('./data/nasdaq_screener.csv')
nasdaq_ticker.sort_values(['Market Cap'],ascending=False)
fifty_largest= nasdaq_ticker.nlargest(50,'Market Cap')
twenty_largest= nasdaq_ticker.nlargest(20,'Market Cap')
ticker_list_twenty = twenty_largest['Symbol'].to_list()
ticker_list = tuple(ticker_list_twenty)

In [29]:
def get_company_financials(ticker_list):
    all_financials = {}
    if not ticker_list:
         return "Please provide atleast one ticker"
    else: 
        # avoid looping through dataframes due to loc index issues
        ticker_list = list(ticker_list)
        for tick in ticker_list:
            ticker_report ={}
            dict_financials  = yf.get_financials(tick,yearly =True, quarterly = False)
            ticker_report["income_statement"]= dict_financials['yearly_income_statement'].fillna(0)
            ticker_report["balance_sheet"] = dict_financials['yearly_balance_sheet'].fillna(0)
            ticker_report["yearly_cash_flow"] = dict_financials['yearly_cash_flow'].fillna(0)
            all_financials[tick] = ticker_report
    return all_financials

In [30]:
def get_company_performance(ticker_list):

    all_performance = {}
    if not ticker_list:
         return "Please provide atleast one ticker"
    else: 
        ticker_list = list(ticker_list)
        for tick in ticker_list:
            performance = {}

            try:
                dividend = yf.get_dividends(tick)
                performance["dividend"] = dividend
                earnings = yf.get_earnings_history(tick)
                performance["earnings"] = pd.DataFrame(earnings)
                all_performance[tick] = performance
                # catch exceptions raised on no data available
            except Exception as e : 
                ticker_list.remove(tick)
                #print(f"No dividend for {tick}") 
                
    return all_performance  



In [17]:
def get_financial_stats(ticker_list):
    all_fin_stats = {}
    if not ticker_list:
         return "Please provide atleast one ticker"
    else:
        ticker_list = list(ticker_list) 
        for tick in ticker_list:
            try:
                fin_stats = yf.get_stats(tick)
                all_fin_stats[tick] = fin_stats
            except Exception as e:
                print(e)
                #ticker_list.remove(tick)
    return all_fin_stats

In [32]:
all_financials={}
all_performance={}
while  not (bool(all_financials)):
    all_financials = get_company_financials(ticker_list)



In [38]:
while not (bool(all_performance)):
    all_performance = get_company_performance(ticker_list)

In [19]:
all_financial_stats={}
while not (bool(all_financial_stats)):
    all_financial_stats = get_financial_stats(ticker_list)

In [23]:
beta_stats = {}
df_Beta= pd.DataFrame(columns=['BETA'],index= ["Ticker"])
for tick in all_financial_stats.keys():
    fin_df = all_financial_stats[tick]
    if ("Attribute" in fin_df.columns):
        fin_attr = fin_df.set_index("Attribute")
        beta = fin_attr.loc["Beta (5Y Monthly)"].values[0]
        #print(f"here {tick}")
        #df_Beta.index.append(tick)
        df_Beta.loc[tick]=beta
        #df_Beta.at[tick,"BETA"] = beta
    else:
        print (f"Skipped {tick}") 
        # 
df_Beta.dropna(inplace=True)


In [34]:
list_dfs_FCFE  = []
list_dfs_ND2EB = []
cf_labels = ['netBorrowings','capitalExpenditures','depreciation']
is_labels = ["netIncome",'ebit']

df_ratios = pd.DataFrame()
for tick in all_financials.keys():
    stock_ratios ={}
    yearly_cash_flow = all_financials[tick]["yearly_cash_flow"]
    income_stmnt = all_financials[tick]["income_statement"]
    if (cf_labels[0] in yearly_cash_flow.index) and (cf_labels[1] in yearly_cash_flow.index):
         capex = yearly_cash_flow.loc[cf_labels[1]]
         net_borrow = yearly_cash_flow.loc[cf_labels[0]]
         depreciation = yearly_cash_flow.loc[cf_labels[2]]
    else: 
        #print(tick)
        del all_financials[tick]    
    if is_labels[0] in income_stmnt.index:
        net_income = income_stmnt.loc["netIncome"]
        ebit = income_stmnt.loc["ebit"]
    else: 
        #print(tick)
        del all_financials[tick]
    FCFE = net_income - capex + net_borrow
    ND_to_EB = net_borrow / (ebit + depreciation)
    #print(f"Ticker {tick} has FCFE of {FCFE}")
    df_FCFE = pd.DataFrame(FCFE).transpose()
    df_ND2EB = pd.DataFrame(ND_to_EB).transpose()
    
    df_ND2EB.columns = df_ND2EB.columns.year
    df_ND2EB['Ticker'] = pd.Series(tick, index=df_ND2EB.index)
    list_dfs_ND2EB.append(df_ND2EB)
    
    df_FCFE.columns = df_FCFE.columns.year
    df_FCFE['Ticker'] = pd.Series(tick, index=df_FCFE.index)
    list_dfs_FCFE.append(df_FCFE)
   

In [35]:
#concat and clean
df_all_ND2EB = pd.concat(list_dfs_ND2EB).set_index("Ticker")
df_all_ND2EB = df_all_ND2EB.drop(columns=(min(df_all_ND2EB.columns) + 4),axis=1,errors='ignore')
df_all_FCFE = pd.concat(list_dfs_FCFE).set_index("Ticker")
df_all_FCFE= df_all_FCFE.drop(columns=(min(df_all_FCFE.columns) + 4),axis=1,errors='ignore')
df_all_ND2EB.columns.name= "year"
df_all_FCFE.columns.name= "year"

In [42]:
all_div_payout= []
all_div_coverage=[]

for tick in all_performance.keys():
    
    divi = all_performance[tick]['dividend']
    ann_total_divi = divi.groupby(divi.index.year).sum()
    ann_total_divi = ann_total_divi.T
    ann_total_divi = ann_total_divi[ann_total_divi.columns.intersection(df_all_FCFE.columns)]
   
    earnings = all_performance[tick]['earnings']
    eps = earnings[["startdatetime","epsactual"]]
    eps.loc[:, ("startdatetime")] = pd.to_datetime(eps.loc[:, ("startdatetime")], errors="coerce",format="%Y-%m-%d")
    eps_annual = eps.groupby(eps.loc[:, ("startdatetime")].dt.year).sum()
    eps_annual = eps_annual.T
    eps_annual = eps_annual[eps_annual.columns.intersection(df_all_FCFE.columns)]
    div_coverage = pd.DataFrame(eps_annual.loc['epsactual'] / ann_total_divi.loc["dividend"]).T
    div_coverage["Ticker"] = tick
    div_coverage =div_coverage.set_index("Ticker")
    all_div_coverage.append(div_coverage.dropna())
   
    div_payout =  pd.DataFrame(ann_total_divi.loc["dividend"] / eps_annual.loc['epsactual']).T
    div_payout["Ticker"] = tick
    div_payout =div_payout.set_index("Ticker")
    all_div_payout.append(div_payout.dropna())
   

In [43]:
#concat and clean
df_div_payout = pd.concat(all_div_payout)
df_div_payout.columns.name= "year"
df_div_coverage= pd.concat(all_div_coverage)
df_div_coverage.columns.name= "year"

In [44]:
df_div_payout['mean'] = df_div_payout.mean(axis=1)

In [45]:
df_div_payout_ordered = df_div_payout.sort_values(by=['mean'], ascending= False)
df_div_payout_ordered = df_div_payout_ordered.reset_index()

In [46]:
df_div_coverage['mean'] = df_div_coverage.mean(axis=1)

In [47]:
df_div_coverage_ordered = df_div_coverage.sort_values(by=['mean'], ascending= False)
df_div_coverage_ordered = df_div_coverage_ordered.reset_index()
df_div_coverage_ordered

year,Ticker,2017,2018,2019,2020,mean
0,AAPL,14.959350,13.744681,3.921053,4.049536,9.168655
1,CMCSA,4.261603,2.625272,4.714286,3.155556,3.689179
2,ASML,3.955624,3.958091,1.718223,2.095305,2.931810
3,MSFT,2.125786,2.430233,2.640212,2.961722,2.539488
4,CSCO,2.115044,2.140625,2.304348,2.181818,2.185459
5,AVGO,3.328482,1.827193,1.902679,1.658427,2.179195
6,TXN,1.948113,2.060837,1.676012,1.422043,1.776751
7,PEP,1.616162,1.524101,1.465858,1.366799,1.493230


In [48]:
df_all_FCFE['mean'] = df_all_FCFE.mean(axis=1)

In [50]:
df_all_ND2EB['mean'] = df_all_ND2EB.mean(axis=1)
#df_all_ND2EB


year,2020,2019,2018,2017,mean
Ticker,,,,,
AAPL,0.032310,-0.102240,0.005281,0.405785,0.085284
MSFT,-0.084555,-0.073315,-0.226901,0.847244,0.115618
AMZN,-0.022928,-0.278120,-0.276853,0.637064,0.014790
GOOG,0.175965,-0.005568,-0.001466,-0.002401,0.041633
GOOGL,0.175965,-0.005568,-0.001466,-0.002401,0.041633
FB,-0.014671,-0.022317,0.017107,0.021526,0.000411
TSLA,-0.582261,0.367403,0.054434,846.250000,211.522394
NVDA,1.539510,-0.003935,-0.238193,NaN,0.432461
PYPL,0.923399,0.866315,0.351262,0.302583,0.610890


In [51]:
df_with_divi_FCFE = df_all_FCFE[df_all_FCFE.index.isin(df_div_payout.index)].sort_values(by=['mean'], ascending= False)
# df_with_divi_FCFE["Rank"]= lambda t: .strftime('%d-%b-%Y'), inplace=True)
df_with_divi_FCFE = df_with_divi_FCFE.reset_index()
df_with_divi_FCFE

year,Ticker,2020,2019,2018,2017,mean
0,AAPL,6.721900e+10,5.793200e+10,7.327600e+10,8.981600e+10,7.206075e+10
1,MSFT,5.420400e+10,4.916500e+10,1.800200e+10,6.507700e+10,4.661200e+10
2,CMCSA,2.107800e+10,1.913800e+10,5.832700e+10,3.555300e+10,3.352400e+10
3,PEP,2.265300e+10,1.119100e+10,8.835000e+09,9.876000e+09,1.313875e+10
4,AVGO,1.112600e+10,1.639000e+10,1.192100e+10,6.519000e+09,1.148900e+10
5,TXN,7.242000e+09,6.605000e+09,7.711000e+09,4.851000e+09,6.602250e+09
6,CSCO,1.794000e+09,1.144600e+10,-7.056000e+09,1.589900e+10,5.520750e+09
7,ASML,5.998700e+09,3.355100e+09,3.162800e+09,2.162600e+09,3.669800e+09


In [52]:
df_with_divi_ND2EB = df_all_ND2EB[df_all_ND2EB.index.isin(df_div_payout.index)].sort_values(by=['mean'], ascending= False)
df_with_divi_ND2EB = df_with_divi_ND2EB.reset_index()
df_with_divi_ND2EB

year,Ticker,2020,2019,2018,2017,mean
0,AVGO,0.690542,1.305257,-0.101693,0.506060,0.600041
1,CMCSA,-0.004347,-0.146370,1.205437,0.115390,0.292527
2,MSFT,-0.084555,-0.073315,-0.226901,0.847244,0.115618
3,TXN,0.145566,0.110961,0.131718,0.068795,0.114260
4,PEP,0.855336,-0.027441,-0.553814,0.160507,0.108647
5,AAPL,0.032310,-0.102240,0.005281,0.405785,0.085284
6,ASML,0.333258,-0.001204,-0.000827,-0.085162,0.061516
7,CSCO,-0.644040,-0.068022,-0.540687,0.363450,-0.222325


In [53]:
ranked_dfs = []
df_with_divi_ND2EB.name="ND2EB"
df_with_divi_FCFE.name="FCFE"
df_div_coverage_ordered.name="div_coverage"
df_div_payout_ordered.name="div_payout"
ranked_dfs.append(df_with_divi_ND2EB)
ranked_dfs.append(df_with_divi_FCFE)
ranked_dfs.append(df_div_coverage_ordered)
ranked_dfs.append(df_div_payout_ordered)


#### P1 add all the ranks and select the lowest 5 and add government bond
#### P2  add all and select five
#### P3 dividend payout ration - 20% 1 share  , grwoing company eps  40% 4 shares, 40 % - i tB
#### P4 stable 10% as p1 1number, hish risk based on beta 65 % 3number  t bill 5 %, crypto 20 1number 
### p5 crpto 45%   2 number , beta 50% 3 number , Tbill 5 % 1
###  Crypto bitcoin 
###  Ethereum
###  portfolio and weights 

In [59]:
available_stocks = df_div_payout.index.values
available_stocks

array(['AAPL', 'MSFT', 'ASML', 'CMCSA', 'CSCO', 'AVGO', 'PEP', 'TXN'],
      dtype=object)

In [60]:
available_stocks = df_div_payout.index.values
score_each={}
score_each_debug={}
for df in ranked_dfs:
    for tick in available_stocks:
       if tick not in score_each:
            score_each[tick] = 0             
       score_each[tick] = score_each[tick] + df[df["Ticker"] == tick ].index.values[0]
       

In [61]:
score_each

{'AAPL': 12,
 'MSFT': 10,
 'ASML': 20,
 'CMCSA': 10,
 'CSCO': 20,
 'AVGO': 11,
 'PEP': 14,
 'TXN': 15}

In [62]:
#lowest score is best performer in each table 
score_sort = sorted(score_each.items(), key =  lambda kv:(kv[1], kv[0]))

In [63]:
score_sort

[('CMCSA', 10),
 ('MSFT', 10),
 ('AVGO', 11),
 ('AAPL', 12),
 ('PEP', 14),
 ('TXN', 15),
 ('ASML', 20),
 ('CSCO', 20)]

In [64]:
# 10 year US give bond = ^TNX
ten_year_bond_symbol = "^TNX"
# top 2 crypto 
bitcoin = "BTC-USD"
etherum = "ETH-USD"

In [65]:
# Stable Dividend growth company – 20% T-Bill (Govt. Bonds) – 80% 
p1_instruments = []
weight_instruments ={} 
five_stocks =score_sort[0:5] 
weight_instruments[ten_year_bond_symbol] = 0.8
for st in five_stocks:
    weight= (0.2 /5)
    weight_instruments[st[0]] = weight

p1_instruments.append(weight_instruments)

In [66]:
p2_instruments = []
weight_instruments ={} 
five_stocks =score_sort[0:5] 
weight_instruments[ten_year_bond_symbol] = 0.65


for st in five_stocks:
    weight= (0.35 /5)
    weight_instruments[st[0]] = weight

p2_instruments.append(weight_instruments)

In [67]:
p3_instruments = []
weight_instruments ={} 
four_stocks =score_sort[0:4] 
weight_instruments[ten_year_bond_symbol] = 0.4
# 1 share at 20% 
best_div_payout = df_div_payout_ordered.head(1).loc[0:1,"Ticker"].values[0]
weight_instruments[best_div_payout] = 0.2

for st in four_stocks:
    weight= (0.4 /5)
    weight_instruments[st[0]] = weight

p3_instruments.append(weight_instruments)


In [68]:
p3_instruments

[{'^TNX': 0.4,
  'PEP': 0.2,
  'CMCSA': 0.08,
  'MSFT': 0.08,
  'AVGO': 0.08,
  'AAPL': 0.08}]

In [69]:
df_Beta.sort_values(by="BETA", ascending=False).head(3).index

Index(['TSLA', 'PDD', 'NVDA'], dtype='object')

In [70]:
# P4 stable 10% as p1 1number, hish risk based on beta 65 % 3number  t bill 5 %, crypto 20 1number

In [71]:
p4_instruments = []
weight_instruments ={} 
one = score_sort[0:1][0][0] 
weight_instruments[one] = 0.1 
weight_instruments[ten_year_bond_symbol] = 0.05
# 1 share at 20% 
best_div_payout = df_div_payout_ordered.head(1).loc[0:1,"Ticker"].values[0]
weight_instruments[best_div_payout] = 0.2
weight_instruments[bitcoin] = 0.2
for st in (df_Beta.sort_values(by="BETA", ascending=False).head(3).index):
    weight= (0.65 / 3)
    weight_instruments[st] = weight

p4_instruments.append(weight_instruments)

In [72]:
p4_instruments

[{'CMCSA': 0.1,
  '^TNX': 0.05,
  'PEP': 0.2,
  'BTC-USD': 0.2,
  'TSLA': 0.21666666666666667,
  'PDD': 0.21666666666666667,
  'NVDA': 0.21666666666666667}]

In [73]:
# p5 crpto 45%   2 number , beta 50% 3 number , Tbill 5 % 1
p5_instruments = []
weight_instruments ={} 
weight_instruments[ten_year_bond_symbol] = 0.05
weight_instruments[bitcoin] = (0.45/2)
weight_instruments[etherum] = (0.45/2)
for st in (df_Beta.sort_values(by="BETA", ascending=False).head(3).index):
    weight= (0.5 / 3)
    weight_instruments[st] = weight

p5_instruments.append(weight_instruments)

In [74]:
p5_instruments

[{'^TNX': 0.05,
  'BTC-USD': 0.225,
  'ETH-USD': 0.225,
  'TSLA': 0.16666666666666666,
  'PDD': 0.16666666666666666,
  'NVDA': 0.16666666666666666}]